<a href="https://colab.research.google.com/github/dmlc/gluon-cv/blob/onnx/scripts/onnx/notebooks/action-recognition/inceptionv3_ucf101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install --upgrade onnxruntime

In [ ]:

import numpy as np
import onnxruntime as rt
import urllib.request
import os.path
from PIL import Image
    

In [ ]:

def fetch_model():
    if not os.path.isfile("inceptionv3_ucf101.onnx"):
        urllib.request.urlretrieve("https://apache-mxnet.s3-us-west-2.amazonaws.com/onnx/models/gluoncv-inceptionv3_ucf101-d9cda2fd.onnx", filename="inceptionv3_ucf101.onnx")
    return "inceptionv3_ucf101.onnx"
    
def prepare_img(img_path, input_shape):
    # input_shape: BHWC
    height, width = input_shape[1], input_shape[2]
    img = Image.open(img_path).convert('RGB')
    img = img.resize((width, height))
    img = np.asarray(img)
    img = np.expand_dims(img, axis=0).astype('float32')

    return img
    
def prepare_label():
    from gluoncv.data import UCF101Attr
    return UCF101Attr().classes
    

**Make sure to replace the image you want to use**

In [ ]:

model = fetch_model()
img_path = "Your input"
img = prepare_img(img_path, (1, 299, 299, 3))
label = prepare_label()
    

In [ ]:

# Create a onnx inference session and get the input name
onnx_session = rt.InferenceSession(model, None)
input_name = onnx_session.get_inputs()[0].name    
    

In [ ]:

pred = onnx_session.run([], {input_name: img})[0]
    


(Optional) We use mxnet to process the result.

Feel free to process the result your own way


In [ ]:
!pip3 install --upgrade mxnet

In [ ]:

import mxnet as mx

pred = mx.nd.array(pred)
topK = 5
ind = mx.nd.topk(pred, k=topK)[0].astype('int')
print('The input is classified to be')
for i in range(topK):
    print('	[%s], with probability %.3f.'%
          (label[ind[i].asscalar()], mx.nd.softmax(pred)[0][ind[i]].asscalar()))
    